# Data cleaning

As menthioned in the exploratory analysis notebook, we'll be using a subset of the original dataset (includes data for 45 taxis for the entire year) for subsequent studies. We'll first clean the data.

### Load the dataset
First let's take a quick look at the data:

In [126]:
import pandas as pd

df_raw = pd.read_pickle('subset_data.pkl')
df_raw.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
10,1372636875620000233,C,NaN,NaN,20000233,1372636875,A,False,"[[-8.619894,41.148009],[-8.620164,41.14773],[-..."
13,1372638595620000233,C,NaN,NaN,20000233,1372638595,A,False,"[[-8.608716,41.153499],[-8.607627,41.153481],[..."
25,1372639092620000233,C,NaN,NaN,20000233,1372639092,A,False,"[[-8.632737,41.168295],[-8.6328,41.16825],[-8...."
63,1372641600620000612,B,NaN,15.0,20000612,1372641600,A,False,"[[-8.585811,41.148666],[-8.585757,41.148972],[..."
64,1372636896620000360,C,NaN,NaN,20000360,1372636896,A,False,"[[-8.617599,41.146137],[-8.617581,41.14593],[-..."


### Missing data
First, we look for rows/entries that contain missing data. The columns 'ORIGIN_CALL' and 'ORIGIN_STAND' contains nan values by definition, so we'll look in columns other than these two. 

In [127]:
# Get list of all column names
column_names = list(df_raw.columns)
# Generate list of all column names except for 'ORIGIN_CALL' and 'ORIGIN_STAND'
filtered_names = [s for s in column_names if s not in ['ORIGIN_CALL', 'ORIGIN_STAND']]

# Get rows that contain missing data
missing_data_rows = df_raw[df_raw[filtered_names].isna().any(axis=1)]
len(missing_data_rows)

0

### Check data format and data type
We can see that there's no rows that contain missing data. Next, let's examine the data to make sure all data are in the correct format or data type. Looking at the summary, and reading the dataset description, 

In [128]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179932 entries, 10 to 1710666
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   TRIP_ID       179932 non-null  int64  
 1   CALL_TYPE     179932 non-null  object 
 2   ORIGIN_CALL   37869 non-null   float64
 3   ORIGIN_STAND  84877 non-null   float64
 4   TAXI_ID       179932 non-null  int64  
 5   TIMESTAMP     179932 non-null  int64  
 6   DAY_TYPE      179932 non-null  object 
 7   MISSING_DATA  179932 non-null  bool   
 8   POLYLINE      179932 non-null  object 
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 12.5+ MB


We can see that columns   
'TRIP_ID', 'TAXI_ID', 'TIMESTAMP' should be int type,  
'CALL_TYPE' and 'DAY_TYPE' should take in one of these three values ['A', 'B', 'C'],  
'ORIGIN_CALL' and 'ORIGIN_STAND' should be float type (the NULL value is represented by _np.nan_, which is also _float_ type),    
'MISSING_DATA' should be bool type,  
'POLYLINE' should be str type.

In [129]:
# Get rows that contain data in incorrect type
# use df.apply and lambda function to check if data in current row is the correct type
incorrect_entries = []
incorrect_entries.append(df_raw[~df_raw['TRIP_ID'].apply(lambda x: isinstance(x, int))])
incorrect_entries.append(df_raw[~df_raw['TAXI_ID'].apply(lambda x: isinstance(x, int))])
incorrect_entries.append(df_raw[~df_raw['TIMESTAMP'].apply(lambda x: isinstance(x, int))])

incorrect_entries.append(df_raw[~df_raw['CALL_TYPE'].isin(['A', 'B', 'C'])])
incorrect_entries.append(df_raw[~df_raw['DAY_TYPE'].isin(['A', 'B', 'C'])])

incorrect_entries.append(df_raw[~df_raw['ORIGIN_CALL'].apply(lambda x: isinstance(x, float))])
incorrect_entries.append(df_raw[~df_raw['ORIGIN_STAND'].apply(lambda x: isinstance(x, float))])

incorrect_entries.append(df_raw[~df_raw['MISSING_DATA'].apply(lambda x: isinstance(x, bool))])
incorrect_entries.append(df_raw[~df_raw['POLYLINE'].apply(lambda x: isinstance(x, str))])

In [130]:
size_incorrect_entries = [len(d) for d in incorrect_entries]
size_incorrect_entries

[0, 0, 0, 0, 0, 0, 0, 0, 0]

### GPS coordinates data
We can see that all data are in correct data type. Next, we want to remove POLYLINE data that doesn't have practical meaning, which are POLYLINE data that doesn't contain any GPS coordinates, or contains only one coordinate. The POLYLINE data is a string in this format: '[[LONGITUDE, LATITUDE],[LONGITUDE, LATITUDE],...]]', note that the GPS coordinates has a precision of six decimal places, so the longest lenght of a POLYLINE string with one coordinate is 25.

In [131]:
polyline_records_to_remove = df_raw[df_raw['POLYLINE'].apply(len) <= 25]
polyline_records_to_remove.tail()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
1708287,1404139515620000432,C,NaN,NaN,20000432,1404139515,A,False,"[[-8.644968,41.158647]]"
1708464,1404139454620000188,C,NaN,NaN,20000188,1404139454,A,False,"[[-8.659098,41.160402]]"
1709057,1404148899620000129,B,NaN,53.0,20000129,1404148899,A,False,"[[-8.613918,41.141142]]"
1710413,1404107068620000618,C,NaN,NaN,20000618,1404107068,A,False,"[[-8.526681,41.164506]]"
1710643,1386603894620000970,C,NaN,NaN,20000970,1386603894,A,False,[]


In [132]:
# Remove entries from the raw data where the POLYLINE column is not meaningful
df = df_raw[df_raw['POLYLINE'].apply(len) > 25]

print("Removed {} rows.".format(len(polyline_records_to_remove)))

Removed 5267 rows.


### Remove duplicates
Next, we remove duplicated rows in the dataset. Apart from finding duplicated rows, we also need to find duplicates in trip ID since it should be an unique identifier for each trip. Also, it is impossible for one taxi to start two trips at the same time, so we need to find rows that have duplicated values in taxi ID and timestamp simultaneously. It's unlikely for two trips to have exactly the same list GPS coordinates, so we should also remove duplicates in POLYLINE.

In [133]:
# Find all occurrences of duplicate rows (including the first one)
all_duplicates = df[df.duplicated(keep=False)]
len(all_duplicates)

0

In [134]:
# Find all duplicates in trip ID
duplicates_tripid = df[df['TRIP_ID'].duplicated(keep=False)]

In [135]:
duplicates_tripid

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
1145967,1394118360620000066,C,NaN,NaN,20000066,1394118360,A,False,"[[-8.633178,41.163867],[-8.633583,41.163318]]"
1146242,1394118360620000066,C,NaN,NaN,20000066,1394118360,A,False,"[[-8.633565,41.163309],[-8.633241,41.16285],[-..."
1357574,1398185157620000066,C,NaN,NaN,20000066,1398185157,A,False,"[[-8.637525,41.159394],[-8.637246,41.159277]]"
1357714,1398185157620000066,C,NaN,NaN,20000066,1398185157,A,False,"[[-8.636715,41.159187],[-8.636031,41.159106],[..."


There're two pairs of duplicates in trip ID. By examining each pair, we can see that for the first pair of data, the first entry includes POLYLINE that only has two GPS coordinates, while the other entry includes GPS coordinates of a complete trip. Therefore, we can keep only the second entry as the data for this trip ID. Similarly, we also keep the second entry for the second pair.

In [136]:
# Remove desired rows
df = df.drop([list(duplicates_tripID.index)[0], list(duplicates_tripID.index)[2]])

Remove duplicates in POLYLINE:

In [137]:
# Find all duplicates in POLYLINE
duplicates_polyline = df[df['POLYLINE'].duplicated(keep=False)]
len(duplicates_polyline)

35

By carefully examining these data, we can see that the duplicates of a POLYLINE is made by the same taxi on different timestamps. We can keep the first entry of the duplicated entries.

In [138]:
df = df.drop_duplicates(subset='POLYLINE', keep='first')

Find duplicates in taxi ID and timestamp:

In [139]:
# Find duplicates in taxi ID and timestamp
duplicates_taxiid_timestamp = df[df.duplicated(subset=['TAXI_ID', 'TIMESTAMP'], keep=False)]
len(duplicates_taxiid_timestamp)

0

### Missing data in GPS coordinates

In [140]:
# Check how many entries of GPS data stream have one (or more) locations that are missing
true_count = len(df[df['MISSING_DATA'] == True])
true_count

1

In [141]:
row_missing_GPS_data = df[df['MISSING_DATA'] == True]
row_missing_GPS_data

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
105621,1374554455620000625,B,NaN,23.0,20000625,1374554455,A,True,"[[-8.612559,41.145975],[-8.612577,41.145975],[..."


### Save relevant data
To improve efficiency and reduce computational overhead, we will save only the columns that are relevant for the next steps in our study, which is trip ID (we'll use this as the unique identifier for each trip) and POLYLINE.   
Since all entries in 'MISSING_DATA' takes the value FALSE except for one, we'll remove this column, and keep the index of the entry with missing GPS coordinates for later reference.

In [142]:
# Index of the row that has missing GPS coordinates
index_missing_GPS_data = list(row_missing_GPS_data.index)[0]

# Save only 'TRIP_ID' and 'POLYLINE' column
df_clean = df[['TRIP_ID', 'POLYLINE']]

# Save the cleaned data as .pkl file
df_clean.to_pickle('clean_data.pkl')